In [5]:
from google import genai
from google.genai import types
import pathlib
import httpx

client = genai.Client()

# Retrieve and encode the PDF byte
file_path = pathlib.Path("data/le_Petit_Prince_본문_15페이지.pdf")


# Upload the PDF using the File API
sample_file = client.files.upload(
    file=file_path,
)

prompt = """pdf파일에서 텍스트 내용을 추출해줘. pdf는 페이지가 나란히 배치되어 있어. 왼쪽 페이지를 먼저 읽고, 오른쪽 페이지를 읽어와줘.
            그리고 페이지 번호나 책의 본문과 관련 없는 메타 정보는 제거하고 페이지간의 문장을 연결해줘.
        """

response = client.models.generate_content(
    model="gemini-2.5-flash", contents=[sample_file, prompt]
)
print(response.text)

어린 왕자
앙투안 드 생텍쥐페리 지음
송태효 옮김
나는 어린왕자가 철새들의 이동을 따라 떠나왔다고 생각한다.
여기에 실린 그림은 생텍쥐페리 자신이 그린 그림이다.
새로운사람들

「어린왕자」는 생텍쥐페리의 「사람들의 땅」후편이다.
레옹 베르트에게
이 책을 어떤 어른에게 바친 데 대해 아이들에게 용서를 빈다. 내게는 그럴만 한 각별한 사정이 있다. 이 어른이야말로 세상에서 나와 가장 친한 친구이기 때문이다. 또 다른 사정이 있다. 이 어른은 모든 것을, 심지어 아동 도서도 이해할 수 있다. 세 번째 사정은 이렇다. 이 어른이 지금 프랑스에서 굶주린 채 추위에 떨며 살고 있다는 것이다. 이 어른을 위로해 주어야 한다. 이 모든 사정도 성에 차지 않는다면 어린 시절의 그에게 이 책을 바치겠노라. 어른들도 한때는 모두 어린아이였으니까. (하기야 이 사실을 기억하는 어른은 거의 없지만 말이다.) 그래서 헌사를 이렇게 수정하노라.
어린아이 시절의
레옹 베르트에게
I
여섯 살 시절 나는 『모험기』라는 제목의 원시림 이야기책에서 멋진 그림 하나를 본 적이 있다. 맹수를 삼키는 보아 뱀 그림이었다. 위의 그림은 그걸 옮겨 그려 본 것이다.

그 책에는 이렇게 쓰여 있었다. “보아 뱀은 먹이를 씹지도 않고 통째로 삼킨다. 그러고는 꼼짝달싹 못 한 채 먹이가 소화될 때까지 여섯 달 동안 잠만 잔다.”
그래서 정글 속 모험에 대해 곰곰이 생각해 보았다. 그러고는 혼자서 색연필로 내 생애 첫 번째 그림을 그려 내는 쾌거를 이루었다. 내 그림 1호는 이러했다.
이 걸작을 어른들에게 보여 주고 그림이 무섭지 않은지 물었다. 어른들은 “모자가 뭐가 무서운데?”라고 답했다.
내 그림은 모자를 그린 게 아니었다. 코끼리를 소화하는 보아 뱀 그림이었다. 그래서 나는 어른들이 알아볼 수 있도록 보아 뱀 속을 그렸다. 어른들에게는 언제나 설명이 필요한 법이다. 나의 그림 2호는 이러했다.
어른들은 속이 보이거나 보이지 않거나 보아 뱀 그림 따위는 집어치우고 차라리 지리와 역사와 산수와 문법에 관심을

In [11]:
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트를 문장 단위로 나누기 위한 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 각 chunk의 최대 크기
    chunk_overlap=100,  # chunk 간 겹치는 부분
    separators=["\n\n", "\n", ".", "!", "?", ";", ",", " ", ""]  # 문장 단위로 나누기 위한 구분자
)

# response.text를 chunk로 나누기
text_chunks = text_splitter.split_text(response.text)

# Document 객체 리스트로 변환
docs = [Document(page_content=chunk) for chunk in text_chunks]

print(f"총 {len(docs)}개의 문서로 분할되었습니다.")
for i, doc in enumerate(docs):  # 처음 3개만 미리보기
    print(doc.page_content)

총 11개의 문서로 분할되었습니다.
어린 왕자
앙투안 드 생텍쥐페리 지음
송태효 옮김
나는 어린왕자가 철새들의 이동을 따라 떠나왔다고 생각한다.
여기에 실린 그림은 생텍쥐페리 자신이 그린 그림이다.
새로운사람들
「어린왕자」는 생텍쥐페리의 「사람들의 땅」후편이다.
레옹 베르트에게
이 책을 어떤 어른에게 바친 데 대해 아이들에게 용서를 빈다. 내게는 그럴만 한 각별한 사정이 있다. 이 어른이야말로 세상에서 나와 가장 친한 친구이기 때문이다. 또 다른 사정이 있다. 이 어른은 모든 것을, 심지어 아동 도서도 이해할 수 있다. 세 번째 사정은 이렇다. 이 어른이 지금 프랑스에서 굶주린 채 추위에 떨며 살고 있다는 것이다. 이 어른을 위로해 주어야 한다. 이 모든 사정도 성에 차지 않는다면 어린 시절의 그에게 이 책을 바치겠노라. 어른들도 한때는 모두 어린아이였으니까. (하기야 이 사실을 기억하는 어른은 거의 없지만 말이다.) 그래서 헌사를 이렇게 수정하노라.
어린아이 시절의
레옹 베르트에게
I
여섯 살 시절 나는 『모험기』라는 제목의 원시림 이야기책에서 멋진 그림 하나를 본 적이 있다. 맹수를 삼키는 보아 뱀 그림이었다. 위의 그림은 그걸 옮겨 그려 본 것이다.
그 책에는 이렇게 쓰여 있었다. “보아 뱀은 먹이를 씹지도 않고 통째로 삼킨다. 그러고는 꼼짝달싹 못 한 채 먹이가 소화될 때까지 여섯 달 동안 잠만 잔다.”
그래서 정글 속 모험에 대해 곰곰이 생각해 보았다. 그러고는 혼자서 색연필로 내 생애 첫 번째 그림을 그려 내는 쾌거를 이루었다. 내 그림 1호는 이러했다.
이 걸작을 어른들에게 보여 주고 그림이 무섭지 않은지 물었다. 어른들은 “모자가 뭐가 무서운데?”라고 답했다.
내 그림은 모자를 그린 게 아니었다. 코끼리를 소화하는 보아 뱀 그림이었다. 그래서 나는 어른들이 알아볼 수 있도록 보아 뱀 속을 그렸다. 어른들에게는 언제나 설명이 필요한 법이다. 나의 그림 2호는 이러했다.
어른들은 속이 보이거나 보이지 않거나 보아 뱀 그림 따위는 집어치우고 차라리 지

In [10]:
text_chunks[1]

'「어린왕자」는 생텍쥐페리의 「사람들의 땅」후편이다.\n레옹 베르트에게\n이 책을 어떤 어른에게 바친 데 대해 아이들에게 용서를 빈다. 내게는 그럴만 한 각별한 사정이 있다. 이 어른이야말로 세상에서 나와 가장 친한 친구이기 때문이다. 또 다른 사정이 있다. 이 어른은 모든 것을, 심지어 아동 도서도 이해할 수 있다. 세 번째 사정은 이렇다. 이 어른이 지금 프랑스에서 굶주린 채 추위에 떨며 살고 있다는 것이다. 이 어른을 위로해 주어야 한다. 이 모든 사정도 성에 차지 않는다면 어린 시절의 그에게 이 책을 바치겠노라. 어른들도 한때는 모두 어린아이였으니까. (하기야 이 사실을 기억하는 어른은 거의 없지만 말이다.) 그래서 헌사를 이렇게 수정하노라.\n어린아이 시절의\n레옹 베르트에게\nI\n여섯 살 시절 나는 『모험기』라는 제목의 원시림 이야기책에서 멋진 그림 하나를 본 적이 있다. 맹수를 삼키는 보아 뱀 그림이었다. 위의 그림은 그걸 옮겨 그려 본 것이다.'

In [12]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import pathlib
import httpx

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
 
client = genai.Client()
 
# Retrieve and encode the PDF byte
file_path = pathlib.Path("data/le_Petit_Prince_본문_15페이지.pdf")
 
 
# Upload the PDF using the File API
sample_file = client.files.upload(
    file=file_path,
)

prompt = """pdf파일에서 텍스트 내용을 추출해줘. pdf는 페이지가 나란히 배치되어 있어. 왼쪽
      페이지를 먼저 읽고, 오른쪽 페이지를 읽어와줘.
      그리고 페이지 번호나 책의 본문과 관련 없는 메타 정보는 제거하고 페이지간의 문장을
      연결해줘.
        """

response = client.models.generate_content(
    model="gemini-1.5-flash", contents=[sample_file, prompt]
)
print(response.text)


어린 왕자

앙투안 드 생텍쥐페리 지음
송태효 옮김

나는 어린왕자가 철새들의 이동을 따라 떠나왔다고 생각한다.
여기에 실린 그림은 생텍쥐페리 자신이 그린 그림이다.

레옹 베르트에게
이 책을 어떤 어른에게 바친 데 대해 아이들에게 용서를 빈다. 내게는 그럴만 한 각별한 사정이 있다. 내게는 그럴만한 각별한 사정이 있다. 이 어른이야말로 세상에서 나와 가장 친한 친구이기 때문이다. 또 다른 사정이 있다. 이 어른은 모든 것을, 심지어 아동 도서도 이해할 수 있다. 세 번째 사정은 이렇다. 이 어른이 지금 프랑스에서 굶주린 채 추위에 떨며 살고 있다는 것이다. 이 어른을 위로해 주어야 한다. 이 모든 사정도 성에 차지 않는다면 어린 시절의 그에게 이 책을 바치겠노라. 어른들도 한때는 모두 어린아이였으니까. (하기야 이 사실을 기억하는 어른은 거의 없지만 말이다.) 그래서 헌사를 이렇게 수정하노라.

어린아이 시절의
레옹 베르트에게


여섯 살 시절 나는 『모험기』라는 제목의 원시림 이야기책에서 멋진 그림 하나를 본 적이 있다. 맹수를 삼키는 보아 뱀 그림이었다. 위의 그림은 그걸 옮겨 그려 본 것이다. 그 책에는 이렇게 쓰여 있었다. “보아 뱀은 먹이를 씹지도 않고 통째로 삼킨다. 그러고는 꼼짝달싹 못 한 채 먹이가 소화될 때까지 여섯 달 동안 잠만 잔다.” 그래서 정글 속 모험에 대해 곰곰이 생각해 보았다. 그러고는 혼자서 색연필로 내 생애 첫 번째 그림을 그려 내는 쾌거를 이루었다. 내 그림 1호는 이러했다. 이 걸작을 어른들에게 보여 주고 그림이 무섭지 않은지 물었다. 어른들은 “모자가 뭐가 무서운데?”라고 답했다. 내 그림은 모자를 그린 게 아니었다. 코끼리를 소화하는 보아 뱀 그림이었다. 그래서 나는 어른들이 알아볼 수 있도록 보아 뱀 속을 그렸다. 어른들에게는 언제나 설명이 필요한 법이다. 나의 그림 2호는 이러했다. 어른들은 속이 보이거나 보이지 않거나 보아 뱀 그림 따위는 집어치우고 차라리 지리와 역사와 산수와 문법에 관심을 가지라고 충고했다. 이